In [3]:
import requests
import pandas as pd
import plotly.express as px

# 1. Hent eksportdata
def hent_eksportdata():
    url = "https://data.ssb.no/api/v0/no/table/03024/"
    query = {
        "query": [],
        "response": {"format": "json-stat2"}
    }
    r = requests.post(url, json=query)
    data = r.json()

    from itertools import product
    dims = list(data["dimension"].keys())
    values = data["value"]
    dim_vals = [list(data["dimension"][d]["category"]["label"].values()) for d in dims]
    combos = list(product(*dim_vals))
    df = pd.DataFrame(combos, columns=dims)
    df["Value"] = values
    df_pivot = df.pivot_table(index="Tid", columns=["VareGrupper2", "ContentsCode"],
                               values="Value", aggfunc="sum").reset_index()
    df_pivot.columns = ["År og ukenr.", "Fersk laks - Kilospris", "Fersk laks - Vekt (tonn)",
                        "Frosset laks - Kilospris", "Frosset laks - Vekt (tonn)"]
    df_pivot["UkeID"] = df_pivot["År og ukenr."].str.replace("U", "-")
    return df_pivot

# 2. Interaktiv visualisering med Plotly
def vis_interaktiv_laks():
    df = hent_eksportdata()
    df = df.dropna()

    # Konverter uke til dato (mandag i uken)
    df["UkeDato"] = pd.to_datetime(df["UkeID"] + "-1", format="%Y-%W-%w")

    # Velg ut kun relevante kolonner
    df_plot = df[["UkeDato", "Fersk laks - Kilospris", "Frosset laks - Kilospris"]].copy()

    # Gjør datasettet "langt" for interaktivt plotting
    df_melt = df_plot.melt(id_vars="UkeDato", var_name="Produkt", value_name="Pris (kr/kg)")

    # Lag interaktiv linjegraf
    fig = px.line(
        df_melt,
        x="UkeDato",
        y="Pris (kr/kg)",
        color="Produkt",
        title="Interaktiv utvikling i kilospris: Fersk vs. Frosset laks",
        labels={"UkeDato": "Dato"}
    )

    fig.update_layout(
        hovermode="x unified",
        template="plotly_white"
    )

    fig.show()

# 3. Kjør
if __name__ == "__main__":
    vis_interaktiv_laks()
